In [1]:
from controllables.energyplus import (
    World,
    #WeatherModel,
    #Report,
    Actuator,
    OutputVariable,
)

from energyplus.dataset.basic import dataset as _epds_

world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / '1ZoneEvapCooler.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.Specs.Output(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    runtime=World.Specs.Runtime(
        #recurring=True,
        #design_day=True,
    ),
)

In [3]:
world.add('logging:progress').awaitable.run()

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-5' coro=<World.run() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/core/utils/awaitables.py:81>>

### Barebone

In [2]:
from controllables.core.tools.gymnasium import (
    Agent,
    DictSpace,
    BoxSpace,
)

import numpy as _numpy_


agent = Agent({
    'action_space': DictSpace({
        'thermostat': BoxSpace(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='MAIN ZONE',
        ))
    }),    
    'observation_space': DictSpace({
        'temperature': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(world[OutputVariable.Ref(
            type='Zone Air Temperature',
            key='MAIN ZONE',
        )]),
    }),
}).__attach__(world)

In [4]:
for _ in range(10):
    ctx = world.events['end_zone_timestep_after_zone_reporting'].wait()
    print(ctx)

CallbackContext(input=CallbackArguments(Context(event=Event(ref='end_zone_timestep_after_zone_reporting')), ), output=<controllables.core.specs.callbacks.CallbackResult object at 0x7fa5d05bb890>)
CallbackContext(input=CallbackArguments(Context(event=Event(ref='end_zone_timestep_after_zone_reporting')), ), output=<controllables.core.specs.callbacks.CallbackResult object at 0x7fa5d05bb650>)
CallbackContext(input=CallbackArguments(Context(event=Event(ref='end_zone_timestep_after_zone_reporting')), ), output=<controllables.core.specs.callbacks.CallbackResult object at 0x7fa5d05bb590>)
CallbackContext(input=CallbackArguments(Context(event=Event(ref='end_zone_timestep_after_zone_reporting')), ), output=<controllables.core.specs.callbacks.CallbackResult object at 0x7fa5d05bbc10>)
CallbackContext(input=CallbackArguments(Context(event=Event(ref='end_zone_timestep_after_zone_reporting')), ), output=<controllables.core.specs.callbacks.CallbackResult object at 0x7fa5d05bbdd0>)
CallbackContext(inpu

In [5]:
for _ in range(100):
    ctx = await world.events['end_zone_timestep_after_zone_reporting'].awaitable(deferred=True)
    try: 
        print(world['wallclock'].value, agent.observe())
    except Exception as e: 
        ctx.ack()
        raise e
    agent.act({'thermostat': 20})
    ctx.ack()

2024-07-21 00:11:00 {'temperature': array(22.434202, dtype=float32)}
2024-07-21 00:25:00 {'temperature': array(23.254333, dtype=float32)}
2024-07-21 00:31:25.714286 {'temperature': array(22.179026, dtype=float32)}
2024-07-21 00:40:00 {'temperature': array(21.829006, dtype=float32)}
2024-07-21 00:53:20 {'temperature': array(22.748838, dtype=float32)}
2024-07-21 01:03:20 {'temperature': array(22.092676, dtype=float32)}
2024-07-21 01:11:15 {'temperature': array(21.604664, dtype=float32)}
2024-07-21 01:20:00 {'temperature': array(22.797531, dtype=float32)}
2024-07-21 01:35:00 {'temperature': array(23.166058, dtype=float32)}
2024-07-21 01:45:00 {'temperature': array(20.270628, dtype=float32)}
2024-07-21 01:50:00 {'temperature': array(20.614998, dtype=float32)}
2024-07-21 02:03:20 {'temperature': array(21.791773, dtype=float32)}
2024-07-21 02:12:30 {'temperature': array(22.170866, dtype=float32)}
2024-07-21 02:25:00 {'temperature': array(22.271648, dtype=float32)}
2024-07-21 02:35:00 {'tempe

### `gymnasium` `Env`s

In [6]:
from controllables.core.tools.gymnasium import Env

env = Env({
    'action_space': DictSpace({
        'thermostat': BoxSpace(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='MAIN ZONE',
        ))
    }),    
    'observation_space': DictSpace({
        'temperature': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(world[OutputVariable.Ref(
            type='Zone Air Temperature',
            key='MAIN ZONE',
        )]),
    }),
    'reward_function': lambda agent: 1,
    'events': {'step': 'end_zone_timestep_after_zone_reporting'},
}).__attach__(world)

In [7]:
from controllables.core import TemporaryUnavailableError

for _ in range(100):
    try: print('step', env.step(env.action_space.sample()))
    except TemporaryUnavailableError: print('.')

step StepResult(observation={'temperature': array(20., dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(18.951159, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(19.010223, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(18.566862, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.70477, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.530119, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.36541, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(19.778484, dtype=float32)}, reward

In [8]:
world.stop()